In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2, os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image, ImageCms

https://www.kaggle.com/datasets/arnaud58/landscape-pictures/data
https://learnopencv.com/convolutional-neural-network-based-image-colorization-using-opencv/
https://www.tensorflow.org/tutorials/load_data/images?hl=pl
https://www.kaggle.com/code/theblackmamba31/autoencoder-grayscale-to-color-image

https://www.kaggle.com/code/basu369victor/image-colorization-basic-implementation-with-cnn
https://richzhang.github.io/colorization/
#Duperele

In [34]:
def set_seed(seed=2137):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

#Stałe dla wczytywania obrazów
IMAGE_DIM = (128,128)
DATA_PATH='data/landscape'
BATCH_SIZE = 32
COLOR_MODE = 'rgb'
LABELS = None
SHUFFLE=True

#Funkcje pomocnicze
#Funkcja do zamiany obrazu z rgb space do lab space
def img_to_lab(img_batch):
    def convert(img):
        img = img.numpy().astype(np.uint8)
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        L = lab[:, :, 0:1] / 255.0  # 0–1
        ab = (lab[:, :, 1:] - 128) / 128.0  # -1 to 1
        return L.astype(np.float32), ab.astype(np.float32)
    L, ab = tf.numpy_function(convert, [img_batch], [tf.float32, tf.float32])
    L.set_shape((IMAGE_DIM[0], IMAGE_DIM[1], 1))
    ab.set_shape((IMAGE_DIM[0], IMAGE_DIM[1], 2))
    return L, ab, rgb

        

In [31]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH,
    labels=LABELS
    ,image_size=IMAGE_DIM,
    color_mode=COLOR_MODE,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
)


Found 4319 files.


135

In [32]:
dataset = dataset.map(lambda x:img_to_lab(x),num_parallel_calls=tf.data.AUTOTUNE)

135